Collecting cases from Yahoo! Finance with days with no market manipulation
The script collects close cases of the same companies so that non-manipulated days were not overlayed

In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
import yfinance as yf
import string

n = 20
p = int(n * 7)
m = 0.45

path = ""

mmf = pd.read_excel(path + "Market Manipulation Cases\\Market Manipulation Cases.xlsx")

mmf['Interval'] = mmf['End of MM'] - mmf['Start of MM'] + timedelta(days = 1)
mmf['SideInterval'] = mmf['Interval']*(1-m)/(2*m) + timedelta(days = 2)

mmf['Restriction on Start'] = False
mmf['Restriction on End'] = False

mmf['Start'] = ''
mmf['End'] = ''


for i in mmf.index:
    row = mmf.loc[i]

    if (not pd.isnull(row['Start Restriction'])):
        if row['Start Restriction'] > row['Start of MM'] - row['SideInterval'] - timedelta(days = p):
            row['Restriction on Start'] = True
            row['Start'] = row['Start Restriction']
    else:
        row['Start'] = row['Start of MM'] - row['SideInterval'] - timedelta(days = p)
    
    if (not pd.isnull(row['End Restriction'])):
        if row['End Restriction'] < row['End of MM'] + row['SideInterval']:
            row['Restriction on End'] = True
            row['End'] = row['End Restriction']
    else:
        row['End'] = row['End of MM'] + row['SideInterval']
        
    mmf.loc[i] = row

list_of_names = []
list_of_company_cases = []
    
for i in mmf.index:
    name = mmf.loc[i, 'Name']
    
    if (not name in list_of_names) and mmf.loc[i, 'To include'] == 1:
        df = mmf.loc[mmf['Name'] == name]
        df = df.sort_values(by = 'Start')
        df.reset_index(drop=True, inplace=True)
        list_of_company_cases.append(df)
        
    list_of_names.append(name)
    

separated_cases_by_name = []
    
for company_cases in list_of_company_cases:
    
    separated_cases = []
    
    separated_case = [] 
    
    for ind in company_cases.index:
        company_case = company_cases.loc[ind]
#         print(company_case)
        
        if ind == 0:
            separated_case = [company_case, company_case.to_frame().transpose()]
        else:
            
            to_merge = False
            
            dif = ''
            
            if company_case['Restriction on Start'] == True:
                if company_case['Start'] <= separated_case[0]['End']:
                    print('Here!')
                    to_merge = True
                    dif = separated_case[0]['End'] - company_case['Start']
            else:
                if company_case['Start'] + timedelta(days = p) <= separated_case[0]['End']:
                    to_merge = True
                    dif = separated_case[0]['End'] - company_case['Start'] - timedelta(days = p)
            
            
            
            if to_merge:
                if separated_case[0]['End'] > company_case['End']:
                    separated_case = [separated_case[0], pd.concat([separated_case[1], company_case.to_frame().transpose()])]
                else:
                    cases_of_separated_case = separated_case[1]
                    par_of_separated_case = separated_case[0]

                    cases_of_separated_case = pd.concat([cases_of_separated_case, company_case.to_frame().transpose()])

                    right_dif = 0
                    left_dif = 0

                    if pd.isnull(par_of_separated_case['Start Restriction']):
                        left_dif = 2 * dif
                    else:
                        left_dif = par_of_separated_case['Start'] - dif - par_of_separated_case['Start Restriction']

                    if pd.isnull(company_case['End Restriction']):
                        right_dif = 2 * dif
                    else:
                        right_dif = company_case['End Restriction'] - company_case['End'] - dif

                    par_of_separated_case['End Restriction'] = company_case['End Restriction']

                    par_of_separated_case['End'] = company_case['End'] + min(dif/2 + max(timedelta(days = 0), left_dif - dif / 2), right_dif)
                    par_of_separated_case['Start'] = par_of_separated_case['Start'] - min(dif/2 + max(timedelta(days = 0), right_dif - dif / 2), left_dif)

                    separated_case = [par_of_separated_case, cases_of_separated_case]
            else:

                separated_cases.append(separated_case)
                separated_case = [company_case, company_case.to_frame().transpose()]
    
    separated_cases.append(separated_case)
    separated_cases_by_name.append([separated_case[0]['Name'], separated_cases])
    
files_by_name = []
    
for separated_cases in separated_cases_by_name:
    name = separated_cases[0]
    
    files = []
    
    for i in range(0, len(separated_cases[1])):
        separated_case = separated_cases[1][i]
        
        print(name)
        
        if name == 'GRO' or name == 'QBA' or name == 'ORG.V':
            print(separated_case)
            
        data = yf.download(name, separated_case[0]['Start'] - timedelta(days = 1), separated_case[0]['End'] + timedelta(days = 1))
        
        manipulated_list = []
        
        for x in data.index:
            manipulated = False
            for case_ind in separated_case[1].index:
                case = separated_case[1].loc[case_ind]
                if x  >= case['Start of MM'] and x <= case['End of MM']:
                    manipulated = True
            manipulated_list.append(manipulated)
            
        data['Manipulated'] = manipulated_list
        
        indeces_to_delete = data[ data['Volume'] == 0 ].index
        data.drop(indeces_to_delete, inplace = True)
        
        
        days_before_manip = len(data.loc[:separated_case[0]['Start of MM']])
    
        
        data.reset_index(inplace=True)
        
        if days_before_manip > n + separated_case[0]['SideInterval'].days:
            data.drop(range(0, days_before_manip - (n + separated_case[0]['SideInterval'].days)), inplace = True)
                               
        string = path + "Raw Separate Daily Data\\" + name + "_" + str(i) + ".xlsx"
        files.append(string)
        data.to_excel(string, index = False)
    files_by_name.append([name, files])

C:\Users\Matvei\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Matvei\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Matvei\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Matvei\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

RJF
[*********************100%***********************]  1 of 1 completed
COF
[*********************100%***********************]  1 of 1 completed
SUSS
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SUSS: No data found for this date range, symbol may be delisted
CHTR
[*********************100%***********************]  1 of 1 completed
ARSN
[*********************100%***********************]  1 of 1 completed
KVMD
[*********************100%***********************]  1 of 1 completed
GRO
[Name                                    GRO
Start of MM             2013-03-07 00:00:00
End of MM               2013-04-23 00:00:00
Start Restriction                       NaN
End Restriction         2013-04-23 00:00:00
To include                                1
Appearance                                1
Interval                   48 days 00:00:00
SideInterval               31 days 08:00:00
Restriction on Start                  False
Restriction on End         

Multiple ways to represent data for ML algorithms

In [ ]:
#Normalization without variance
def data_standartization_var1(comp, n):
    cols = ['Open','High', 'Low', 'Close', 'Adj Close', 'Volume', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            sets = []
            
            for l in [0, 1, 2, 3, 4, 5]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            
            for prices in sets:
                mean = prices.mean()
                std = prices.std
                
                if std == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - mean) / (4 * std)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Normalization with variance
def data_standartization_var2(comp, n):
    cols = ['Volume', 'Open','High', 'Low', 'Close', 'Adj Close', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            
            for l in [0, 1, 2, 3, 4]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from adj close
def data_standartization_var4(comp, n): 
    cols = ['Volume','Adj close', 'Open dif', 'High dif', 'Low dif', 'Close dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            for l in [4]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2, 3]:
                sets.append((comp.iloc[d:j + 1, l] - comp.iloc[d:j + 1, 4]).reset_index(drop=True))
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from close
def data_standartization_var3(comp, n): 
    cols = ['Volume','Open', 'Close', 'Open dif', 'High dif', 'Low dif', 'Adj dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            for l in [0, 3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2, 4]:
                sets.append((comp.iloc[d:j + 1, l] - comp.iloc[d:j + 1, 3]).reset_index(drop=True))
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from adj close with linear regression

from sklearn import linear_model

def data_standartization_var6(comp, n): 
    cols = ['Volume','Close', 'Open dif', 'High dif', 'Low dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            
            for l in [3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2]:
                reg = linear_model.LinearRegression()
                reg.fit(comp.iloc[d:j + 1, 3].values.reshape(-1, 1), comp.iloc[d:j + 1, l].values.reshape(-1, 1), comp.iloc[d:j + 1, 5].array)
                reseted = comp.iloc[d:j + 1, l].reset_index(drop=True)
                sets.append(reseted - pd.DataFrame(list(reg.predict(comp.iloc[d:j + 1, 3].values.reshape(-1, 1)))).iloc[:, 0])
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from adj close with linear regression withut weights

from sklearn import linear_model

def data_standartization_var7(comp, n): 
    cols = ['Volume','Close', 'Open dif', 'High dif', 'Low dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            
            for l in [3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2]:
                reg = linear_model.LinearRegression()
                reg.fit(comp.iloc[d:j + 1, 3].values.reshape(-1, 1), comp.iloc[d:j + 1, l].values.reshape(-1, 1))
                reseted = comp.iloc[d:j + 1, l].reset_index(drop=True)
                sets.append(reseted - pd.DataFrame(list(reg.predict(comp.iloc[d:j + 1, 3].values.reshape(-1, 1)))).iloc[:, 0])
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            if row[1:7] == [0, 0, 0, 0, 0, 0]:
                print(j)
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Normalization of percentage change without variance
def data_standartization_var8(comp, n):
    cols = ['Volume', 'Open','High', 'Low', 'Close', 'Adj Close', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            sets = []
            
            for l in [0, 1, 2, 3, 4, 5]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            
            for i in sets:
                prices = i
                prev_prices = prices.iloc[0:n].reset_index(drop=True)
                new_prices = prices.iloc[1:n+1].reset_index(drop=True)
                
                prices = (new_prices - prev_prices)*100 / prev_prices
                
                sum_of_prices = prices.sum(axis=0)
                average_price = sum_of_prices/(n+1)
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                sum_of_prices_minus_average_squared = prices_minus_average_squared.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared / n
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((prices.iloc[n-1] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Normalization of percentages with variance
def data_standartization_var9(comp, n):
    cols = ['Volume', 'Change of Volume', 'Open','High', 'Low', 'Close', 'Adj Close', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            
            for l in [0, 1, 2, 3, 4, 5]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            
            for i in sets:
                prices = i
                prev_prices = prices.iloc[0:n].reset_index(drop=True)
                new_prices = prices.iloc[1:n+1].reset_index(drop=True)
                
                prices = (new_prices - prev_prices) / prev_prices
                
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((prices.iloc[n-1] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from close, adj close is not weighted
def data_standartization_var5(comp, n): 
    cols = ['Adj dif', 'Volume', 'Close', 'Open dif', 'High dif', 'Low dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            
            sets1 = []
            sets1.append((comp.iloc[d:j + 1, 4] - comp.iloc[d:j + 1, 3]).reset_index(drop=True))
            for l in [5]:
                sets1.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for volumes in sets1:
                sum_of_volumes = volumes.sum(axis=0)
                average_volume = sum_of_volumes / (n+1)
                
                volumes_minus_average = volumes - average_volume
                volumes_minus_average_squared = volumes_minus_average ** 2
                sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
                sample_variance = sum_of_volumes_minus_average_squared / n
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((volumes.iloc[n] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            for l in [3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2]:
                sets.append((comp.iloc[d:j + 1, l] - comp.iloc[d:j + 1, 3]).reset_index(drop=True))
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from close without var
def data_standartization_var10(comp, n): 
    cols = ['Volume','Open', 'Close', 'Open dif', 'High dif', 'Low dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            for l in [0, 3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2]:
                sets.append((comp.iloc[d:j + 1, l] - comp.iloc[d:j + 1, 3]).reset_index(drop=True))
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            
            for i in sets:
                prices = i
                prices_by_volumes = prices
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/(n+1)
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / n
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
#Differecnes with variance from adj close with linear regression

from sklearn import linear_model

def data_standartization_var11(comp, n): 
    cols = ['Volume','Close', 'Open dif', 'High dif', 'Low dif', 'High Low dif', 'Manipulated']
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
    
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            
            sets = []
            
            for l in [3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2]:
                reg = linear_model.LinearRegression()
                reg.fit(comp.iloc[d:j + 1, 3].values.reshape(-1, 1), comp.iloc[d:j + 1, l].values.reshape(-1, 1), comp.iloc[d:j + 1, 5].array)
                reseted = comp.iloc[d:j + 1, l].reset_index(drop=True)
                sets.append(reseted - pd.DataFrame(list(reg.predict(comp.iloc[d:j + 1, 3].values.reshape(-1, 1)))).iloc[:, 0])
            sets.append((comp.iloc[d:j + 1, 1] - comp.iloc[d:j + 1, 2]).reset_index(drop=True))
            
            for i in sets:
                prices = i
                prev_prices = prices.iloc[0:n].reset_index(drop=True)
                new_prices = prices.iloc[1:n+1].reset_index(drop=True)
                
                
                
                prices_by_volumes = prices
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/(n+1)
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / n
                
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))

            row.append(int(comp.iloc[j, 6]))
            
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

In [ ]:
def data_standartization_var12(comp, n): 
    cols = ['Volume', 'Volume variance',
            
            'Open with volume', 'Open with volume variance', 'High with volume', 'High with volume variance', 'Low with volume',
            'Low with volume variance', 'Close with volume', 'Close with volume variance',
            
            'Open minus close with volume', 'Open minus close with volume variance', 'High minus close with volume',
            'High minus close with volume variance', 'Low minus close with volume', 'Low minus close with volume variance',
            
            'Volume percentage change', 'Volume percentage change variance',
            
            'Open percentage change', 'Open percentage change variance', 'High percentage change',
            'High percentage change variance', 'Low percentage change', 'Low percentage change variance',
            'Close percentage change', 'Close percentage change variance',
            
#             'Open minus percentage change close', 'Open minus close percentage change variance',
#             'High minus close percentage change', 'High minus close percentage change variance', 
#             'Low minus close percentage change', 'Low minus close percentage change variance',
            'Manipulated']
    
    newcomp = pd.DataFrame(columns = cols)
    
    for j in comp.index:
        if j > n - 1:
            d = j - n
            
            row = []
            
            volumes = comp.iloc[d:j + 1, 5].reset_index(drop=True)
            sum_of_volumes = volumes.sum(axis=0)
            average_volume = sum_of_volumes / (n+1)
            
            volumes_minus_average = volumes - average_volume
            volumes_minus_average_squared = volumes_minus_average ** 2
            sum_of_volumes_minus_average_squared = volumes_minus_average_squared.sum(axis=0)
            sample_variance = sum_of_volumes_minus_average_squared / n
            
            if sample_variance == 0:
                row.append(0.0)
            else:
                row.append((comp.iloc[j, 5] - average_volume) / (4 * math.sqrt(sample_variance)))
            row.append(sample_variance)
            
            sets = []
            
            for l in [0, 1, 2, 3]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for l in [0, 1, 2]:
                sets.append((comp.iloc[d:j + 1, l] - comp.iloc[d:j + 1, 3]).reset_index(drop=True))
            for i in sets:
                prices = i
                prices_by_volumes = prices * volumes
                sum_of_prices_by_volume = prices_by_volumes.sum(axis=0)
                average_price = sum_of_prices_by_volume/sum_of_volumes
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                prices_minus_average_squared_by_volumes = prices_minus_average_squared * volumes
                sum_of_prices_minus_average_squared_by_volumes = prices_minus_average_squared_by_volumes.sum(axis=0)
                sample_variance = sum_of_prices_minus_average_squared_by_volumes / sum_of_volumes
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((i.iloc[n] - average_price) / (4 * math.sqrt(sample_variance)))
                row.append(sample_variance)
            
            sets = []
            
            for l in [0, 1, 2, 3, 5]:
                sets.append(comp.iloc[d:j + 1, l].reset_index(drop=True))
            for i in sets:
                prices = i
                
                prev_prices = prices.iloc[0:n].reset_index(drop=True)
                new_prices = prices.iloc[1:n+1].reset_index(drop=True)
                
                prices = (new_prices - prev_prices)*100 / prev_prices
                
                sum_of_prices = prices.sum(axis=0)
                average_price = sum_of_prices/ n
                
                prices_minus_average = prices - average_price
                prices_minus_average_squared = prices_minus_average**2
                
                sum_of_prices_minus_average_squared = prices_minus_average_squared.sum(axis=0)
                
                sample_variance = sum_of_prices_minus_average_squared / (n - 1)
                
                if sample_variance == 0:
                    row.append(0.0)
                else:
                    row.append((prices.iloc[n-1] - average_price) / (4 * math.sqrt(sample_variance)))
                row.append(sample_variance)

            row.append(int(comp.iloc[j, 6]))
            
            newcomp.loc[len(newcomp)] = row
    return newcomp

End of variants. Combinig all days after deriving new way to present their data

In [ ]:
#normalizaing and combining data

import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

def company_data_standartization(comp, n, standartization_var):
    if standartization_var == 1:
        return data_standartization_var1(comp, n)
    if standartization_var == 2:
        return data_standartization_var2(comp, n)
    if standartization_var == 3:
        return data_standartization_var3(comp, n)
    if standartization_var == 4:
        return data_standartization_var4(comp, n)
    if standartization_var == 5:
        return data_standartization_var5(comp, n)
    if standartization_var == 6:
        return data_standartization_var6(comp, n)
    if standartization_var == 7:
        return data_standartization_var7(comp, n)
    if standartization_var == 8:
        return data_standartization_var8(comp, n)
    if standartization_var == 9:
        return data_standartization_var9(comp, n)
    if standartization_var == 10:
        return data_standartization_var10(comp, n)
    if standartization_var == 11:
        return data_standartization_var11(comp, n)
    if standartization_var == 12:
        return data_standartization_var12(comp, n)
    
def new_all_data_standartization(n, standartization_var):
    
    all_stand_cases = []
    
    for [name, files] in files_by_name:
        
        stand_cases = []
        
        for file in files:
            case = pd.read_excel(file)
            if len(case) > n:
                stand_case = company_data_standartization(case.drop('Date', axis=1), n, standartization_var)
                
                stand_case['Company'] = [name]*len(stand_case)
                
                dates = pd.DataFrame(case.loc[n:, 'Date'])
                dates.reset_index(inplace=True)
                stand_case['Date'] = dates['Date']
                stand_cases.append(stand_case)
        
        if len(stand_cases) != 0:
            stand_cases_united = pd.concat(stand_cases)
            stand_cases_united.sort_values('Manipulated', ascending=False, inplace = True)
            stand_cases_united.drop_duplicates(subset = 'Date', keep = "first", inplace = True)
            all_stand_cases.append(stand_cases_united)
        
    all_stand_cases_united = pd.concat(all_stand_cases)
    all_stand_cases_united.reset_index(drop = True, inplace=True)
    
    string = path + "All Normalized Data\\Var" + str(standartization_var) + "\\" + "AllDays" + ".xlsx"
    all_stand_cases_united.to_excel(string, index = False)      
    
    manip_days = pd.DataFrame(columns = all_stand_cases_united.columns)
    not_manip_days = pd.DataFrame(columns = all_stand_cases_united.columns)

    for i in all_stand_cases_united.index:
        if all_stand_cases_united.loc[i, 'Manipulated'] == 1:
            manip_days.loc[len(manip_days)] = all_stand_cases_united.loc[i]
        else:
            not_manip_days.loc[len(not_manip_days)] = all_stand_cases_united.loc[i]

    manip_days.reset_index(drop = True, inplace = True)
    not_manip_days.reset_index(drop = True, inplace = True)

    manip_days.to_excel("All Normalized Data\\Var" + str(standartization_var) + "\\" + "Manipulated Days" + ".xlsx", index = False)
    not_manip_days.to_excel("All Normalized Data\\Var" + str(standartization_var) + "\\" + "Not Manipulated Days" + ".xlsx", index = False)        

In [ ]:
new_all_data_standartization(n, 1)
new_all_data_standartization(n, 2)
new_all_data_standartization(n, 3)
new_all_data_standartization(n, 4)
new_all_data_standartization(n, 5)
new_all_data_standartization(n, 6)
new_all_data_standartization(n, 7)
new_all_data_standartization(n, 8)
new_all_data_standartization(n, 9)
new_all_data_standartization(n, 10)
new_all_data_standartization(n, 11)

In [ ]:
new_all_data_standartization(n, 12)

Ploting histograms for the data

In [ ]:
def data_hist(var):
    string = path + "All Normalized Data\\Var" + str(var) + "\\" + "AllDays" + ".xlsx"
    all_stand_cases_united = pd.read_excel(string, index = False)
    
    for column in all_stand_cases_united.columns.values.tolist()[0:len(all_stand_cases_united.columns) - 3]:
        all_stand_cases_united.loc[:, column].hist(bins=80)
        plt.xlabel(column)
        plt.savefig("Hists\\Var" + str(var) + "\\hist_" + column + ".pdf", format='pdf')
        plt.clf()

In [ ]:
data_hist(1)
data_hist(2)
data_hist(3)
data_hist(4)
data_hist(5)
data_hist(6)
data_hist(7)
data_hist(8)
data_hist(9)
data_hist(10)
data_hist(11)
data_hist(12)

Ploting correlation table for the 12th variant

In [ ]:
import pandas as pd
from sklearn.preprocessing import PowerTransformer

standartization_var = 12
path = ''

string = path + "All Normalized Data\\Var" + str(standartization_var) + "\\" + "AllDays" + ".xlsx"
all_stand_cases_united = pd.read_excel(string, index = False)

pt = PowerTransformer()
pt.fit(all_stand_cases_united.iloc[:, :len(all_stand_cases_united.columns) - 3])
new_data = pt.transform(all_stand_cases_united.iloc[:, :len(all_stand_cases_united.columns) - 3])

new_data.drop(new_data)

cols = all_stand_cases_united.iloc[:, :len(all_stand_cases_united.columns) - 3].columns.values.tolist()

corr = pd.DataFrame(new_data, columns = cols).corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr = corr.mask(corr>=0.7, 1)
corr = corr.mask(corr<0.7, 0)
corr.style.background_gradient(cmap='coolwarm')

Begin of scatter plots

In [ ]:
from itertools import chain, combinations
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt

def scatter_data(standartization_var):
    manipulated = pd.read_excel(path + "All Normalized Data\\Var" + str(standartization_var) + "\\" + "Manipulated Days" + ".xlsx")
    not_manipulated = pd.read_excel(path + "All Normalized Data\\Var" + str(standartization_var) + "\\" + "Not Manipulated Days" + ".xlsx")
    variables = manipulated.columns.values.tolist()
    variables.remove('Manipulated')
    variables.remove('Company')
    variables.remove('Date')
    for combo in list(combinations(variables, 2)):
        ax = manipulated.plot.scatter(x = combo[0], y = combo[1], color = 'Red', label='Manipulated', s=0.01)
        scatter = not_manipulated.plot.scatter(x = combo[0], y = combo[1], color = 'Green', label='Not Manipulated', s=0.01, ax=ax)
        scatter.get_figure().savefig("Scatter\\Var" + str(standartization_var) + "\\scatter_" + combo[0] + "+" + combo[1] + ".pdf", format='pdf')
        plt.clf()
    

In [ ]:
scatter_data(1)
scatter_data(2)
scatter_data(3)
scatter_data(4)
scatter_data(5)
scatter_data(6)
scatter_data(7)
scatter_data(8)
scatter_data(9)
scatter_data(10)
scatter_data(11)
scatter_data(12)

Begin of analysis

In [ ]:
#function that aproximates the results for a any ML method
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NeighborhoodComponentsAnalysis
import matplotlib.pyplot as plt

def classifier_effectivness(classifier, clas_name, n, standartization_var, remove = [], par = [0], to_print_boxplot = False):
    train_frac = 0.5
    
    s = np.matrix('0.0, 0.0; 0.0, 0.0')
    manip_days = pd.read_excel("All Normalized Data\\Var" + str(standartization_var) + "\\" + "Manipulated Days" + ".xlsx")
    not_manip_days = pd.read_excel("All Normalized Data\\Var" + str(standartization_var) + "\\" + "Not Manipulated Days" + ".xlsx")
    
    manip_index = len(manip_days.columns) - 3
    company_index = manip_index + 1
    
    indexes_to_use = list(range(0, manip_index))
    
    for i in remove:
        indexes_to_use.remove(i)

    distribution = pd.DataFrame(columns=['Sensitivity', 'Specificity', 'Accuracy'])   
    
    for i in range(0, n):
        manip_days = manip_days.sample(frac=1).reset_index(drop = True)
        not_manip_days = not_manip_days.sample(frac=1).reset_index(drop = True)
    
        x_m = manip_days.iloc[:, indexes_to_use]
        x_n = not_manip_days.iloc[:, indexes_to_use].iloc[0:len(x_m)]
        
        y_m = manip_days.iloc[:, manip_index]
        y_n = not_manip_days.iloc[:, manip_index].iloc[0:len(x_m)]

        l_m = int(len(x_m)*train_frac)
        l_n = int(len(x_n)*train_frac)

        x_train = pd.concat([x_m.iloc[:l_m, :], x_n.iloc[:l_n, :]])
        x_test = pd.concat([x_m.iloc[l_m:, :], x_n.iloc[l_n:, :]])
        y_train = pd.concat([y_m.iloc[:l_m], y_n.iloc[:l_n]])
        y_test = pd.concat([y_m.iloc[l_m:], y_n.iloc[l_n:]])
    
        x_test.reset_index(drop = True, inplace = True)
        y_test.reset_index(drop = True, inplace = True)
        x_train.reset_index(drop = True, inplace = True)
        y_train.reset_index(drop = True, inplace = True)
    
        y_test=y_test.astype('int')
        y_train=y_train.astype('int')
            
        classifier = classifier.fit(x_train, y_train)
        
        y_pred = classifier.predict(x_test)
        
        new_result = confusion_matrix(y_test, y_pred)
        
        p_hit_m = new_result.item(3)/(new_result.item(3) + new_result.item(2))
        
        p_hit_n = new_result.item(0)/(new_result.item(1) + new_result.item(0))
        
        if to_print_boxplot:
            distribution.loc[len(distribution)] = [p_hit_m, p_hit_n, (p_hit_m+p_hit_n)/2]
        
        s = s + new_result
    
    if to_print_boxplot:
        string = path + 'Boxplots\\' + str(standartization_var) + '\\' + str(clas_name) + '\\' + str(par) + '_' + str(n) + ".pdf"
        distribution.boxplot()
        plt.savefig('string', format='pdf')
    s = s/n
    
    return s
      

In [ ]:
#
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PowerTransformer

data_with_coef = pd.DataFrame(columns = ['Volume', 'Volume variance',
            
            'Open with volume', 'Open with volume variance', 'High with volume', 'High with volume variance', 
                                         'Low with volume',
            'Low with volume variance', 'Close with volume', 
                                         'Close with volume variance',
            
            'Open minus close with volume', 'Open minus close with volume variance', 'High minus close with volume',
                                         'High minus close with volume variance', 
            'Low minus close with volume', 'Low minus close with volume variance',
            
            'Volume percentage change', 'Volume percentage change variance',
            
            'Open percentage change', 'Open percentage change variance', 'High percentage change',
            'High percentage change variance', 'Low percentage change', 'Low percentage change variance',
            'Close percentage change', 'Close percentage change variance', 'Probability'])

# def create_KNN_classifier(par):
#     return KNeighborsClassifier(n_neighbors = par, weights = 'distance')

class NCAClassifier:
    def __init__(self, par):
        self.knn = KNeighborsClassifier(n_neighbors = par[1], weights = 'distance')
        self.nca = NeighborhoodComponentsAnalysis(n_components = par[0], random_state=42, max_iter = 100000, tol = 1e-10000000000, init = 'random')
        
    def fit(self, x_train, y_train):
        self.nca.fit(x_train, y_train) 
        x_train = self.nca.transform(x_train)
        self.knn.fit(x_train, y_train) 
        return self
    def predict(self, x_test):
        x_test = self.nca.transform(x_test)
        return self.knn.predict(x_test)

def create_NCA_classifier(par):
    return NCAClassifier(par)

class NBClassifier:
    def __init__(self):
        self.nb = GaussianNB()
        self.pt = PowerTransformer()
    def fit(self, x_train, y_train):
        self.pt.fit(x_train)
        self.nb.fit(self.pt.transform(x_train), y_train)
        return self
    def predict(self, x_test):
        return self.nb.predict(self.pt.transform(x_test))

def create_NB_classifier(par):
    return NBClassifier()

# def create_DT_classifier(par):
#     return DecisionTreeClassifier()

def create_GB_classifier(par):
    return GradientBoostingClassifier(n_estimators=100000, learning_rate=0.1, random_state=0, loss = "exponential", tol=1e-10000)

class ANNClassifier:
    def __init__(self, par):
        self.mlp = MLPClassifier(alpha = 0.1, hidden_layer_sizes=par, max_iter=200000000, tol=1e-10000000000000, solver='lbfgs', max_fun=15000000000, )
        self.pt = PowerTransformer()
    def fit(self, x_train, y_train):
        self.pt.fit(x_train)
        self.mlp.fit(self.pt.transform(x_train), y_train)
        return self
    def predict(self, x_test):
        return self.mlp.predict(self.pt.transform(x_test))

def create_ANN_classifier(par):
    return ANNClassifier(par)

def create_classifier(clas_name, par):
#     if clas_name == "KNN":
#         return create_KNN_classifier(par)
    if clas_name == "NCA":
        return create_NCA_classifier(par)
    if clas_name == "NB":
        return create_NB_classifier(par)
#     if clas_name == "DT":
#         return create_DT_classifier(par)
    if clas_name == "GB":
        return create_GB_classifier(par)
    if clas_name == "ANN":
        return create_ANN_classifier(par)

In [ ]:
def par_tester(clas_name, n, standartization_var, remove = [], pars = [0], to_print_boxplot = False):
    
    if len(data_with_coef) != 0:
        data_with_coef.drop(data_with_coef.index, inplace=True)
    
    cont = True
    first = True
    
    p = 0.01
    
    res = []
    
    for par in pars:
        classifier = create_classifier(clas_name, par)
        new_result = classifier_effectivness(classifier, clas_name, n, standartization_var, remove, par, to_print_boxplot)
        
        sens = new_result.item(3)/(new_result.item(3) + new_result.item(2))
        
        spes = new_result.item(0)/(new_result.item(1) + new_result.item(0))
        
        res.append([par, [sens, spes]])
        
    return res

Testing parameters:

-For GB:

In [ ]:
set = par_tester('GB', 1, 12)
print(set)

-For ANN:

In [ ]:
pars = [(450), (225, 225), (150, 150, 150)]

In [ ]:
set = par_tester('ANN', 1, 12, [], pars)
print(set)

-For NB:

In [ ]:
par_tester('NB', 10, 12, [2, 3, 4, 5, 6, 8, 9, 11, 13, 17, 23])

-For NCA

In [ ]:
pars = []
for i in range(1, 26):
    for j in range(1, 9):
        pars.append([i, j])

In [ ]:
res = par_tester('NCA', 1, 12, [], pars)

In [ ]:
sensitivity_array = [x[1][0] for x in res]
specificity_array = [x[1][1] for x in res]
accuracy_array = [(x[1][0] + x[1][1])/2 for x in res]

arrays = [sensitivity_array, specificity_array, accuracy_array]

for i in range(3):
    array = arrays[i]
    new_array = np.reshape(array, (25, 8))
    data = pd.DataFrame(new_array)
    data.style.background_gradient(cmap='coolwarm')
    data.to_excel('NCA Parameters\\' + str(i) + ".xlsx")

Testing subsets for NB algorithms

In [ ]:
from itertools import chain, combinations

def almost_powerset(my_list):
    return list(chain.from_iterable(combinations(my_list, r) for r in range(0, len(my_list) - 3)))

def get_the_best_subsets(var, iterations, algorithms, remove = [], number_of_variables):
    number_of_variables = 0
    if var == 12:
        number_of_variables = 26
    elif var == 3:
        number_of_variables = 7
    elif var == 4:
        number_of_variables = 7
    else:
        number_of_variables = 6
    
    my_set = list(range(number_of_variables))
    
    my_set = [x for x in my_set if x not in remove]


    alm_powerset = almost_powerset(my_set)
    
    print(alm_powerset)

    
    for algorithm in algorithms:
        results = []
        for subset in alm_powerset:
            subset = list(subset)
            for x in remove:
                subset.append(x)
            results.append([subset , par_tester(algorithm, iterations, var, subset)])
    
        results.sort(reverse=True, key=lambda x: x[1][0][1][0] + x[1][0][1][1])
        
        new_results = results.copy()

        for x in results:
            if x[1][0][0] <= 0.5 or x[1][0][1] <= 0.5:
                new_results.remove(x)
    return new_results[0:200]

In [ ]:
# [1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 15, 17, 18, 19, 23, 25]
results_nb = get_the_best_subsets(12, 1, ['NB'], [2, 3, 4, 5, 7, 8, 9, 13, 15, 17, 19, 23])

In [ ]:
the_best = []

for [subset, probs] in results_nb[0:100]:
    new_prob = par_tester('NB', 100, 12, subset)
    the_best.append([subset, new_b])
    
the_best.sort(reverse=True, key=lambda x: x[1][1][0] + x[1][1][1])

for i in the_best:
    print(i)

Testing

In [ ]:
class NBClassifier:
    def __init__(self):
        self.nb = GaussianNB()
        self.pt = PowerTransformer()
    def fit(self, x_train, y_train):
        self.pt.fit(x_train)
        self.nb.fit(self.pt.transform(x_train), y_train)
        return self
    def predict(self, x_test):
        return self.nb.predict(self.pt.transform(x_test))

def create_GB_classifier(par):
    return GradientBoostingClassifier(n_estimators=100000, learning_rate=0.01, random_state=0, loss = "exponential", tol=1e-10000)
    
standartization_var = 12

manip_days = pd.read_excel("New All Normalized Data\\Var" + str(standartization_var) + "\\" + "Manipulated Days" + ".xlsx")
not_manip_days = pd.read_excel("New All Normalized Data\\Var" + str(standartization_var) + "\\" + "Not Manipulated Days" + ".xlsx")

# grov = pd.read_excel("INTV.xlsx")
# grov = pd.read_excel("APG.AX.xlsx")
grov = pd.read_excel("0567.HK.xlsx")

# print(grov)

grov = data_standartization_var12(grov, 20)

print(grov)



train_frac = 1   

indexes_to_use = list(range(26))
indexes_to_use = [0, 10, 11, 14, 21]

all_res = pd.DataFrame(columns=['Accuracy', 'Sensitivity', 'Specificity'])

class NCAClassifier:
    def __init__(self, par):
        self.knn = KNeighborsClassifier(n_neighbors = par[1], weights = 'distance')
        self.nca = NeighborhoodComponentsAnalysis(n_components = par[0], random_state=42, max_iter = 100000, tol = 0, init = 'random')
        self.par = par
        
    def fit(self, x_train, y_train):
        self.nca.fit(x_train, y_train)
        
        print(self.nca.n_iter_)
        
        x_train = self.nca.transform(x_train)
        self.knn.fit(x_train, y_train) 
        return self
    def predict(self, x_test):
        x_test = self.nca.transform(x_test)
        return self.knn.predict(x_test)

class ANNClassifier:
    def __init__(self):
        tuple = (225, 225)
        self.mlp = MLPClassifier(activation = 'logistic', alpha = 0.1, hidden_layer_sizes=tuple, max_iter=20000000000, tol=0, solver='lbfgs', max_fun=150000000000000000000, )
        self.pt = PowerTransformer()
    def fit(self, x_train, y_train):
        self.pt.fit(x_train)
        self.mlp.fit(self.pt.transform(x_train), y_train)
        print(self.mlp.n_iter_)
        return self
    def predict(self, x_test):
        return self.mlp.predict(self.pt.transform(x_test))

classifier = ANNClassifier()
# classifier = NCAClassifier([12, 6])
    
n = 1

for i in range(0, n):
        print(i)
        manip_days = manip_days.sample(frac=1).reset_index(drop = True)
        not_manip_days = not_manip_days.sample(frac=1).reset_index(drop = True)
    
        x_m = manip_days.iloc[:, indexes_to_use]
        x_n = not_manip_days.iloc[:, indexes_to_use].iloc[0:len(x_m)]
        
        print(len(x_m))
        print(len(x_n))
        
        y_m = manip_days.iloc[:, 26]
        y_n = not_manip_days.iloc[:, 26].iloc[0:len(x_m)]

#         l_m = int(len(x_m)*train_frac)
#         l_n = int(len(x_n)*train_frac)

        x_train = pd.concat([x_m, x_n])
        y_train = pd.concat([y_m, y_n])
    
        x_test = grov.iloc[:, indexes_to_use]
        y_test = grov.iloc[:, 26]
    
        x_test.reset_index(drop = True, inplace = True)
        y_test.reset_index(drop = True, inplace = True)
        x_train.reset_index(drop = True, inplace = True)
        y_train.reset_index(drop = True, inplace = True)
    
        y_test=y_test.astype('int')
        y_train=y_train.astype('int')
            
        classifier = classifier.fit(x_train, y_train)
        
        y_pred = classifier.predict(x_test)
        
        together = pd.concat([pd.DataFrame(y_pred), pd.DataFrame(y_test)], axis=1)
        print(together)
        
        for ind in together.index:
            if together.loc[ind, 0] != together.iloc[ind, 1]:
                print(ind)
        
        new_result = confusion_matrix(y_test, y_pred)
        
        p_hit_m = new_result.item(3)/(new_result.item(3) + new_result.item(2))
        
        p_hit_n = new_result.item(0)/(new_result.item(1) + new_result.item(0))
        
        all_res.loc[len(all_res)] = [(new_result.item(3) + new_result.item(0))/len(together), p_hit_m, p_hit_n]
print(all_res)